# Read Data

In [7]:
from collections import defaultdict
len_counter = defaultdict(int)

in_path = '/ldev/tensorflow/seq2seq/rawdata/DiaTest/'

message_path = in_path + 'huaweiFull.message'
response_path = in_path + 'huaweiFull.response'

messages = []
responses = []
with open(message_path,'r+',encoding='utf-8') as fin:
    lines = fin.readlines()
    messages = [line.strip('\n') for line in lines]
    for line in messages:
        len_counter[len(line)] += 1

with open(response_path,'r+',encoding='utf-8') as fin:
    lines = fin.readlines()
    responses = [line.strip('\n') for line in lines]
    for line in responses:
        len_counter[len(line)] += 1

assert len(messages) == len(responses)

print('total lines: %d' % (len(messages)))

total lines: 4435959


In [8]:
responses[1000:1010]

['看到 榜样 的 力量 ， 大家 心里 会 更 安心',
 '某 前锋 完全 没有 预判 跑位 ， 速度 还 没有 对手 后卫 转身 加 回 追 速度快',
 '它 这 是 间接 暗示 你 ， 不如 搂 在一起 睡 会儿 吧',
 'I want to go to Yunnan province in 2013 .',
 '如果 雍正 那会儿 有 QQ ， 那 《 甄嬛传 》 里边 的 人 … …',
 '这个 是 日本 的 那个 什么 组合么',
 '位置 是 创意 的 根源 ， 二师兄 ， 哈哈 。',
 '我 爱上 了 一 匹 汗 血 宝马 ， 可是 我 没有 马场 也 没有 钱 。',
 '这个 。 掉 节操 的 问句 ， 床上 咋办 。',
 '要么 带 套 ， 要么 大 肚 。']

# Filter

In [9]:
import re

def filter(text):
    res = re.sub(r'#[\S ]*#',"", text)
    res = re.sub(r'（[\S ]+）',"", text)
    res = re.sub(r'alink',"", res)
    res = re.sub(r'[ ]+'," ", res)
    return res.strip()


for line in responses[1000:1001]:
    print(line)
    print(filter(line) + '#') 

看到 榜样 的 力量 ， 大家 心里 会 更 安心
看到 榜样 的 力量 ， 大家 心里 会 更 安心#


In [10]:
import re
from collections import defaultdict
new_messages = []
new_responses = []
msg_counter = defaultdict(int)
res_counter = defaultdict(int)
str_counter = defaultdict(int)
for msg,res in zip(messages,responses):
    if msg == res:
        continue
    fmsg = filter(msg)
    fres = filter(res)
    vt = fmsg +'alink' + fres
    if msg_counter[fmsg] > 75 or res_counter[fres] > 75 or str_counter[vt] > 20 :
        continue
    if fmsg == fres:
        continue
    msg_counter[fmsg] += 1
    res_counter[fres]  += 1
    str_counter[vt] += 1
    
    nmsg = ' '.join( x for x in fmsg.split())
    nres = ' '.join( x for x in fres.split())
    fmsg = fmsg.split()
    fres = fres.split()
    if len(fmsg) < 2 or len(fres) < 2 or len(fmsg) > 20 or len(fres) > 20:
        continue
    new_messages.append(nmsg.split())
    new_responses.append(nres.split())
print('total lines: %d' % (len(new_messages))) 

total lines: 3788600


In [11]:
seg_messages = []
seg_responses = []
for line in new_messages:
    segs = []
    for item in line:
        if item == '@@':
            segs.append(item)
        else:
            segs = segs + list(item)
    seg_messages.append(segs)

for line in new_responses:
    segs = []
    for item in line:
        if item == '@@':
            segs.append(item)
        else:
            segs = segs + list(item)
    seg_responses.append(segs)


In [12]:
print(seg_messages[0:20])
new_messages = seg_messages
new_responses = seg_responses

[['中', '国', '移', '动', '营', '销', '行', '来', '发', '展', '报', '告'], ['小', '马', '也', '疯', '狂', '-', '-', '-', '-', '-', '-', '地', '位', '之', '争', '。'], ['那', '些', '年', '，', '我', '们', '一', '起', '偷', '看', '过', '的', '电', '视', '。', '「', '暴', '走', '漫', '画', '」'], ['北', '京', '的', '小', '纯', '洁', '们', '，', '周', '日', '见', '。', '#', '硬', '汉', '摆', '拍', '清', '纯', '照', '#'], ['要', '是', '这', '一', '年', '哭', '泣', '的', '理', '由', '不', '再', '是', '难', '过', '而', '是', '感', '动', '会', '多', '么', '好'], ['对', '于', '国', '内', '动', '漫', '画', '作', '者', '引', '用', '工', '笔', '素', '材', '的', '一', '些', '个', '人', '意', '见', '。'], ['猫', '咪', '保', '镖', '最', '赞', '了', '！', '你', '们', '看', '懂', '了', '吗', '？', '！'], ['莫', '愁', '和', '所', '有', '人', '开', '了', '一', '个', '玩', '笑', '—', '—', '其', '实', '，', '她', '是', '会', '正', '常', '唱', '歌', '的', '…', '…'], ['你', '见', '过', '皮', '卡', '丘', '喝', '水', '的', '样', '子', '吗', '？'], ['如', '果', '有', '个', '人', '能', '让', '你', '忘', '掉', '过', '去', '，', '那', 'T', 'A', '很', '可', '能', '就', '是', '你', '的', '未', 

# Statistics

In [13]:
# Statistics
from collections import defaultdict
len_counter = defaultdict(int)
word_len_counter = defaultdict(int)
num_of_words = 0
tokenized_messages = new_messages
tokenized_responses = new_responses
for line in tokenized_messages + tokenized_responses:
    len_counter[len(line)] += 1
    num_of_words += len(line)
    for item in line:
        word_len_counter[len(item)] += 1

sorted_length = sorted(len_counter.items(), key = lambda x:x[0])
total_counter = 0
for lens,counter in sorted_length:
    total_counter += counter
    print('%d %.2f' % (lens, total_counter/len(tokenized_messages)/2))

print('char_nums_per_word:')
sorted_length = sorted(word_len_counter.items(), key = lambda x:x[0])
total_counter = 0
for lens,counter in sorted_length:
    total_counter += counter
    print('%d %.2f' % (lens, total_counter/num_of_words))

2 0.00
3 0.00
4 0.00
5 0.00
6 0.00
7 0.00
8 0.01
9 0.07
10 0.13
11 0.20
12 0.27
13 0.33
14 0.39
15 0.45
16 0.51
17 0.56
18 0.61
19 0.66
20 0.70
21 0.75
22 0.79
23 0.82
24 0.86
25 0.89
26 0.92
27 0.94
28 0.96
29 0.98
30 0.98
31 0.99
32 0.99
33 0.99
34 0.99
35 0.99
36 1.00
37 1.00
38 1.00
39 1.00
40 1.00
41 1.00
42 1.00
43 1.00
44 1.00
45 1.00
46 1.00
47 1.00
48 1.00
49 1.00
50 1.00
51 1.00
52 1.00
53 1.00
54 1.00
55 1.00
56 1.00
57 1.00
58 1.00
59 1.00
60 1.00
61 1.00
62 1.00
63 1.00
64 1.00
65 1.00
66 1.00
67 1.00
68 1.00
69 1.00
70 1.00
71 1.00
72 1.00
73 1.00
74 1.00
75 1.00
76 1.00
77 1.00
78 1.00
79 1.00
82 1.00
83 1.00
char_nums_per_word:
1 1.00


In [18]:
out_path = '/ldev/tensorflow/nmt2/nmt/data/charlevel/'
import random
random.seed(6666)

total_num = len(messages)
dev_num =  20000
test_num = 20000
max_len = 30
train_num = total_num - dev_num - test_num

random_orders = range(0, total_num)
messages = tokenized_messages
responses =  tokenized_responses
file_map = {'message':tokenized_messages, 'response':tokenized_responses}
for file_type in file_map.keys():
    container = file_map[file_type]
    with open(out_path + 'dev.'+file_type,'w+',encoding='utf-8') as fout:
        for i in range(0,dev_num):
            fout.write('%s\n' % ' '.join(container[i][0:max_len]))
    with open(out_path + 'test.'+file_type,'w+',encoding='utf-8') as fout:
        for i in range(dev_num,test_num + test_num):
            fout.write('%s\n' % ' '.join(container[i][0:max_len]))
    with open(out_path + 'train.'+file_type,'w+',encoding='utf-8') as fout:
        for i in range(test_num + test_num,total_num):
            fout.write('%s\n' % ' '.join(container[i][0:max_len]))

# Create Vocab

In [19]:
out_path = '/ldev/tensorflow/nmt2/nmt/data/charlevel/'
in_path = '/ldev/tensorflow/nmt2/nmt/data/charlevel/'

from collections import defaultdict
counter = defaultdict(int)
message_counter = defaultdict(int)
response_counter = defaultdict(int)


for file in ['message','response']:
    with open(in_path + 'train.'+file,'r+',encoding='utf-8') as fin:
        lines = fin.readlines()
        container = message_counter
        if file == 'response':
            container = response_counter
        for line in lines:
            chars = line.strip('\n').split(' ')
            for char in chars:
                if len(char) > 0:
                    counter[char] += 1
                    container[char] += 1

sorted_counter = sorted(counter.items(), key = lambda x:x[1], reverse = True)
sorted_message_counter = sorted(message_counter.items(), key = lambda x:x[1], reverse = True)
sorted_response_counter = sorted(response_counter.items(), key = lambda x:x[1], reverse = True)
print(len(sorted_counter))
print(len(sorted_message_counter))
print(len(sorted_response_counter))
print(sorted_counter[0:100])

10705
6672
10540
[('，', 4756132), ('的', 3989835), ('。', 3599561), ('！', 2090294), ('是', 2016854), ('了', 1976490), ('不', 1657795), ('一', 1637827), ('我', 1466641), ('这', 1339725), ('有', 1219521), ('你', 1124758), ('人', 1117423), ('？', 1115009), ('个', 994458), ('好', 894553), ('么', 839424), ('看', 737735), ('在', 734036), ('大', 714054), ('来', 634469), ('…', 627389), ('就', 620277), ('啊', 605492), ('天', 597183), ('要', 550680), ('都', 546573), ('上', 523847), ('到', 489755), ('哈', 480373), ('#', 469340), ('还', 456149), ('们', 450280), ('子', 444647), ('会', 443480), ('小', 436917), ('没', 430028), ('中', 410926), ('时', 406794), ('真', 404760), ('得', 400609), ('可', 389181), ('说', 387806), ('过', 382557), ('生', 381387), ('最', 374117), ('也', 373604), ('国', 373210), ('能', 372492), ('家', 370674), ('想', 370388), ('下', 368386), ('多', 368216), ('很', 363353), ('年', 353686), ('那', 343895), ('心', 340691), ('0', 340371), ('样', 338569), ('为', 335915), ('去', 328149), ('1', 326782), ('爱', 321009), ('吧', 316864), ('什', 31

In [20]:
out_path = '/ldev/tensorflow/nmt2/nmt/data/charlevel/'
for file in ['message','response']:
    with open(out_path + 'vocab.'+file,'w+',encoding='utf-8') as fout:
        #fout.write('<UNK>\n<S>\n</S>\n');
        for item in sorted_counter:
            fout.write(item[0]+'\n')
with open(out_path + 'vocab.'+'separate.response','w+',encoding='utf-8') as fout:
        #fout.write('<UNK>\n<S>\n</S>\n');
        for item in sorted_response_counter:
            fout.write(item[0]+'\n')
with open(out_path + 'vocab.'+'separate.message','w+',encoding='utf-8') as fout:
        #fout.write('<unk>\n<s>\n</s>\n');
        for i,item in enumerate(sorted_message_counter):
            if len(item[0]) == 0:
                print('flag '+str(i))
            fout.write(item[0]+'\n')

with open(out_path + 'vocab.'+'.counter','w+',encoding='utf-8') as fout:
        fout.write('<unk>\n\t-1\n<s>\t-1\n</s>\t-1\n');
        for item in sorted_counter:
            fout.write('%s\t%d\n'  % (item[0],item[1]))

In [21]:
sums = 0
for i,(lens,counter) in enumerate(sorted_message_counter):
    sums += counter
print(len(sorted_message_counter))
total_counter = 0
for i,(lens,counter) in enumerate(sorted_message_counter):
    total_counter += counter
    if i%10000 == 0:
        print('%d %.4f' % (i, total_counter/sums))

6672
0 0.0366
